1) raw dataset
2) keywords based extraction
3) train/test split
4) sample a specific number of samples (high cardinality)
5) paste the instruction

In [1]:
from huggingface_hub import logout, notebook_login
# logout()
notebook_login()

/data/jaesung/anaconda3/envs/faiss_gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.

In [32]:
## dataset load

from datasets import load_dataset

# qa datasets
medqa = load_dataset("GBaker/MedQA-USMLE-4-options")
medmcqa = load_dataset("openlifescienceai/medmcqa")
pubmedqa = load_dataset("bigbio/pubmed_qa", trust_remote_code=True)
# mfc = load_dataset("veggiebird/medical-flashcards")

# nli datasets
bionli = load_dataset("clinicalnlplab/BioNLI_test")

# information extraction datasets
ddi = load_dataset("YufeiHFUT/DDI2013")
chemdner =load_dataset("kjappelbaum/chemnlp-chemdner")
# medmentions = load_dataset("bigbio/medmentions", trust_remote_code=True) train-108, test-37
# biorel = load_dataset("DFKI-SLT/BioRel")
# phee = load_dataset("sarus-tech/phee")

# generation datasets
healthcaremagic = load_dataset("lighteval/med_dialog", "healthcaremagic")
icliniq = load_dataset("lighteval/med_dialog", "icliniq")
pubmed = load_dataset("ccdv/pubmed-summarization")

In [33]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

tmp = chemdner['train'].to_pandas()
tmp = tmp.drop(columns=["split"])

train_df, temp_df = train_test_split(tmp, test_size=0.2, shuffle=True, random_state=42)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, shuffle=True, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

chemdner = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test" : test_dataset,
})

chemdner

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', '__index_level_0__'],
        num_rows: 15552
    })
    validation: Dataset({
        features: ['entities', 'text', '__index_level_0__'],
        num_rows: 1944
    })
    test: Dataset({
        features: ['entities', 'text', '__index_level_0__'],
        num_rows: 1944
    })
})

In [34]:
from datasets import concatenate_datasets

meddialog = {
    "train": concatenate_datasets([healthcaremagic['train'], icliniq['train']]),
    "validation": concatenate_datasets([healthcaremagic['validation'], icliniq['validation']]),
    "test": concatenate_datasets([healthcaremagic['test'], icliniq['test']]),
}

meddialog

{'train': Dataset({
     features: ['tgt', 'src', 'id'],
     num_rows: 205973
 }),
 'validation': Dataset({
     features: ['tgt', 'src', 'id'],
     num_rows: 25746
 }),
 'test': Dataset({
     features: ['tgt', 'src', 'id'],
     num_rows: 25750
 })}

In [35]:
print("medqa")
print(medqa)
print('------------------------------------------------------------------------------'* 2) 
print("medmcqa")
print(medmcqa)
print('------------------------------------------------------------------------------'* 2) 
print("pubmedqa")
print(pubmedqa)
print('------------------------------------------------------------------------------'* 2) 
print("bionli")
print(bionli)
print('------------------------------------------------------------------------------'* 2) 
print("DDI")
print(ddi)
print('------------------------------------------------------------------------------'* 2)
print("chemdner")
print(chemdner)
print('------------------------------------------------------------------------------'* 2)
print("meddialog")
print(meddialog)
print('------------------------------------------------------------------------------'* 2) 
print("pubmed")
print(pubmed)

medqa
DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
        num_rows: 1273
    })
})
------------------------------------------------------------------------------------------------------------------------------------------------------------
medmcqa
DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 182822
    })
    test: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 6150
    })
    validation: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_na

In [ ]:
import pandas as pd
import os
import json
import ast

keywords = [keyword.lower() for keyword in [
    "Diabetes",
    "Diabetes Mellitus", "Diabetes Mellitus, Experimental", "Diabetes Mellitus, Type 1",
    "Wolfram Syndrome", "Diabetes Mellitus, Type 2", "Diabetes Mellitus, Lipoatrophic",
    "Diabetes, Gestational", "Donohue Syndrome", "Latent Autoimmune Diabetes in Adults",
    "Prediabetic State", "Diabetes Complications", "Diabesity", "Diabetic Angiopathies",
    "Diabetic Cardiomyopathies", "Diabetic Coma", "Diabetic Ketoacidosis",
    "Diabetic Nephropathies", "Diabetic Neuropathies", "Fetal Macrosomia"
]]

def contains_keywords(text, keywords):
    if not text or not isinstance(text, str): 
        return False
    text_lower = text.lower() 
    return any(keyword.lower() in text_lower for keyword in keywords)

def process_dataset_split(dataset_name, split_name, dataset, text_columns, output_file, keywords=["diabetes", "insulin"]):
    rows = []
    for i in range(len(dataset)):
        row = dataset[i]

        for col in text_columns:
            if col in row:
                text = row[col]

            elif col == "abstract" and "passages" in row:
                if isinstance(row["passages"], list):
                    text = " ".join([p["text"] for p in row["passages"] if p.get("type") == "abstract"])
                else:
                    text = ""
            else:
                text = ""

            if not text:
                continue

            is_related = 1 if contains_keywords(text, keywords) else 0

            rows.append({
                "dataset": dataset_name,
                "split_data": split_name,
                "features": row,  
                "input": text,
                "output": is_related
            })

    if rows:  
        df = pd.DataFrame(rows)
        df.to_csv(output_file, mode="a", header=not os.path.exists(output_file), index=False)
        print(f"{dataset_name} ({split_name}): {len(rows)} rows processed and appended to {output_file}")
    else:
        print(f"{dataset_name} ({split_name}): No rows processed.")

datasets = {
    "medqa": {"splits": medqa, "columns": ["question", "answer"]},
    "medmcqa": {"splits": medmcqa, "columns": ["question"]},
    "pubmedqa": {"splits": pubmedqa, "columns": ["QUESTION", "CONTEXTS", "LONG_ANSWER"]},
    "bionli": {"splits": bionli, "columns": ["query", "answer"]},
    "ddi": {"splits": ddi, "columns": ["text"]},
    "chemdner": {"splits": chemdner, "columns": ['text']},
    "meddialog": {"splits": meddialog, "columns": ["tgt", "src"]},
    "pubmed": {"splits": pubmed, "columns": ["abstract"]},
}


output_file = os.path.join("all_data_filtered_by_keywords.csv")

for dataset_name, details in datasets.items():
    splits = details["splits"]
    text_columns = details["columns"] 

    for split_name, split_data in splits.items():
        process_dataset_split(dataset_name, split_name, split_data, text_columns, output_file, keywords=keywords)


medqa (train): 20356 rows processed and appended to all_data_filtered_by_keywords.csv
medqa (test): 2546 rows processed and appended to all_data_filtered_by_keywords.csv
medmcqa (train): 182822 rows processed and appended to all_data_filtered_by_keywords.csv
medmcqa (test): 6150 rows processed and appended to all_data_filtered_by_keywords.csv
medmcqa (validation): 4183 rows processed and appended to all_data_filtered_by_keywords.csv
pubmedqa (train): 600000 rows processed and appended to all_data_filtered_by_keywords.csv
pubmedqa (validation): 33807 rows processed and appended to all_data_filtered_by_keywords.csv
bionli (train): 11088 rows processed and appended to all_data_filtered_by_keywords.csv
bionli (validation): 25612 rows processed and appended to all_data_filtered_by_keywords.csv
bionli (test): 12616 rows processed and appended to all_data_filtered_by_keywords.csv
ddi (train): 18779 rows processed and appended to all_data_filtered_by_keywords.csv
ddi (test): 5761 rows processe

In [1]:
import pandas as pd

all_df = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/all_data_filtered_by_keywords.csv")

In [2]:
# data_filtered_by_keyword
diabetes = all_df[all_df['output'] == 1]

In [3]:
diabetes.head(2)

,dataset,split_data,features,input,output
30,medqa,train,{'question': 'A 68-year-old man presents to th...,A 68-year-old man presents to the emergency de...,1
34,medqa,train,{'question': 'A 68-year-old man comes to the p...,A 68-year-old man comes to the physician becau...,1


In [ ]:
diabetes.groupby(['dataset', 'split_data'])['output'].agg('count')

dataset    split_data
bionli     test            783
           train           594
           validation     1419
chemdner   test             87
           train           706
           validation      104
ddi        test             66
           train           529
           valid           588
meddialog  test           1249
           train          9962
           validation     1145
medmcqa    test             54
           train          1810
           validation       42
medqa      test            132
           train          1129
pubmed     test            468
           train          8061
           validation      479
pubmedqa   train         17558
           validation      981
Name: output, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

def re_train_test_split_unique(dataframe, train_ratio=0.8):
    new_rows = []

    grouped = dataframe.groupby('dataset')

    for dataset_name, group in grouped:
        if dataset_name not in ['medqa', 'medmcqa', 'pubmedqa', 'bionli', 'chemdner', 'ddi', 'meddialog', 'pubmed']:
            continue

        # 중복 제거
        group = group.drop_duplicates(subset='features')

        train_data, test_data = train_test_split(
            group,
            test_size=1-train_ratio,
            random_state=42,
        )

        train_data = train_data.copy()
        train_data['split_data'] = 'train'

        test_data = test_data.copy()
        test_data['split_data'] = 'test'

        new_rows.append(train_data)
        new_rows.append(test_data)

    result_df = pd.concat(new_rows, ignore_index=True)
    return result_df

diabetes2 = re_train_test_split_unique(diabetes, train_ratio=0.85)

In [6]:
diabetes2.groupby(['dataset', 'split_data'])['output'].agg('count')

dataset    split_data
bionli     test           338
           train         1914
chemdner   test           135
           train          762
ddi        test           178
           train         1005
meddialog  test          1615
           train         9151
medmcqa    test           286
           train         1620
medqa      test           185
           train         1048
pubmed     test          1351
           train         7651
pubmedqa   test          1720
           train         9740
Name: output, dtype: int64

In [7]:
import ast
import pandas as pd

def add_input_to_for_sampling(dataframe):
    dataframe = dataframe.copy()
    
    # INPUT 값을 추출하여 새로운 컬럼에 추가
    def extract_for_sampling(row):
        if row['dataset'] == 'medqa':
            features = ast.literal_eval(row['features'])
            input_text = features.get('question', None)
            answer = features.get('answer', None)
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'medmcqa':
            features = ast.literal_eval(row['features'])
            input_text = features.get('question', None)
            answer = ", ".join([
                features.get('opa', ''),
                features.get('opb', ''),
                features.get('opc', ''),
                features.get('opd', ''),
            ]).strip()
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'pubmedqa':
            features = ast.literal_eval(row['features'])
            question = features.get('QUESTION', None)
            answer = features.get('LONG_ANSWER', None)
            return {'input': question, 'output': answer}
        elif row['dataset'] == 'bionli':
            features = ast.literal_eval(row['features'])
            query = features.get('query', '') 
            input_text = query.split("INPUT: ")[1].split("[HYP]")[0].strip() if "INPUT: " in query else None
            answer = features.get('answer', None)
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'chemdner':
            features = ast.literal_eval(row['features'])
            input_text = features.get('text', None)
            answer = features.get('entities', None)
            if isinstance(answer, list) and len(answer) > 0:
                answer = answer[0]
            else:
                answer = ''
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'ddi':
            features = ast.literal_eval(row['features'])
            input_text = features.get('conversations', [])[0].get('value', None).split("INPUT:")[1].split("OUTPUT:")[0].strip()
            answer = features.get('conversations', [])[1].get('value', None)
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'meddialog':
            features = ast.literal_eval(row['features'])
            input_text = features.get('src', None).split("Doctor:")[0].replace("Patient:", "").strip()
            answer = features.get('src', None).split("Doctor:")[1].strip()
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'pubmed':
            features = ast.literal_eval(row['features'])
            input_text = features.get('article', None)
            abstract = features.get('abstract', None)
            return {'input': input_text, 'output': abstract}

    dataframe['for_sampling'] = dataframe.apply(extract_for_sampling, axis=1)

    # chemdner 의 "" 비율 줄이기
    chemdner_df = dataframe[dataframe['dataset'] == 'chemdner'].copy()
    empty_outputs = chemdner_df[chemdner_df['for_sampling'].apply(lambda x: x['output'] == '')]
    non_empty_outputs = chemdner_df[chemdner_df['for_sampling'].apply(lambda x: x['output'] != '')]
    target_empty_size = int(len(non_empty_outputs) * 0.05 / 0.95) 
    sampled_empty_outputs = empty_outputs.sample(n=min(target_empty_size, len(empty_outputs)), random_state=42)
    balanced_chemdner_df = pd.concat([non_empty_outputs, sampled_empty_outputs])
    other_datasets = dataframe[dataframe['dataset'] != 'chemdner']
    final_dataframe = pd.concat([other_datasets, balanced_chemdner_df])
    
    return final_dataframe

diabetes3 = add_input_to_for_sampling(diabetes2)

In [8]:
diabetes3.head(2)

,dataset,split_data,features,input,output,for_sampling
0,bionli,train,"{'id': 'BioNLI17726157', 'query': '\nTASK: Ple...",\nTASK: Please classify the relationship betwe...,1,{'input': '[PRE] Dipeptidyl peptidase IV inhib...
1,bionli,train,"{'id': 'BioNLI2176649', 'query': '\nTASK: Plea...",\nTASK: Please classify the relationship betwe...,1,{'input': '[PRE] Cold exposure reverses the di...


In [9]:
import os
import pandas as pd
import numpy as np
import time
from sentence_transformers import SentenceTransformer
import faiss
from tqdm import tqdm
import random
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def mmr(query_embedding, doc_embeddings, diversity, top_n):
    faiss.normalize_L2(doc_embeddings)
    faiss.normalize_L2(query_embedding.reshape(1, -1))

    selected_indices = []
    candidate_indices = list(range(len(doc_embeddings)))

    for _ in range(top_n):
        if not candidate_indices:
            break

        if len(selected_indices) == 0:
            selected_idx = candidate_indices[np.argmax(np.dot(doc_embeddings[candidate_indices], query_embedding.T))]
        else:
            selected_embeddings = doc_embeddings[selected_indices]
            similarity_to_selected = np.dot(doc_embeddings[candidate_indices], selected_embeddings.T)
            diversity_scores = np.max(similarity_to_selected, axis=1)
            relevance_scores = np.dot(doc_embeddings[candidate_indices], query_embedding.T).flatten()
            mmr_scores = (1 - diversity) * relevance_scores - diversity * diversity_scores
            selected_idx = candidate_indices[np.argmax(mmr_scores)]

        selected_indices.append(selected_idx)
        candidate_indices.remove(selected_idx)

    return selected_indices

def mmr_sampling(dataframe, sampling_dict, embedding_model='all-MiniLM-L6-v2', batch_size=64, diversity=0.7, seed=42):
    set_seed(seed)

    print("Loading SentenceTransformer model on GPU...")
    model = SentenceTransformer(embedding_model, device="cuda:0")

    sampled_rows = []

    for dataset_name, sample_count in tqdm(sampling_dict.items(), desc="Processing datasets"):
        subset = dataframe[(dataframe['dataset'] == dataset_name) & (dataframe['split_data'] == 'train')].copy()
        num_rows = len(subset)
        if num_rows == 0:
            print(f"No data found for dataset: {dataset_name}")
            continue

        print(f"Generating embeddings for dataset: {dataset_name} ({num_rows} rows)")
        start_time = time.time()

        embeddings = model.encode(
            subset['for_sampling'].astype(str).tolist(),
            batch_size=batch_size,
            convert_to_numpy=True,
            show_progress_bar=True
        )

        elapsed_time = time.time() - start_time
        print(f"Embedding generation took {elapsed_time:.2f} seconds for {num_rows} rows")

        query_embedding = np.mean(embeddings, axis=0)

        print(f"Applying MMR for dataset: {dataset_name}")
        selected_indices = mmr(query_embedding, embeddings, diversity, sample_count)
        sampled_subset = subset.iloc[selected_indices]
        sampled_rows.append(sampled_subset)

    return pd.concat(sampled_rows, ignore_index=True)

# 샘플링 설정
sampling_config = {
    'medqa': 1000,
    'medmcqa': 1000,
    'pubmedqa': 8000, # 1000
    'bionli': 1800, # 1500
    'chemdner': 600,
    'ddi': 900,
    'meddialog': 4000,
    'pubmed': 4000,
}

set_seed(42)

# 샘플링 실행
sampled_train = mmr_sampling(diabetes3, sampling_config, batch_size=4096, diversity=0.7, seed=42)

# 결과 확인
print(sampled_train.head())
print(f"총 샘플링된 행 수: {len(sampled_train)}")

# 결과 저장
sampled_train.to_csv("final_combined_train_sample.csv")


/data/jaesung/anaconda3/envs/faiss_gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading SentenceTransformer model on GPU...


Processing datasets:   0%|          | 0/8 [00:00<?, ?it/s]

Generating embeddings for dataset: medqa (1048 rows)


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.39s/it]


...

Generating embeddings for dataset: pubmed (7651 rows)


Batches: 100%|██████████| 2/2 [00:13<00:00,  6.88s/it]


Embedding generation took 21.73 seconds for 7651 rows
Applying MMR for dataset: pubmed


Processing datasets: 100%|██████████| 8/8 [09:59<00:00, 74.89s/it]


  dataset split_data                                           features  \
0   medqa      train  {'question': 'A 42-year-old woman presents to ...   
1   medqa      train  {'question': "A 69-year-old man presents to hi...   
2   medqa      train  {'question': 'A 64-year-old female with a long...   
3   medqa      train  {'question': 'An endocrinologist is working wi...   
4   medqa      train  {'question': "A pediatrician is investigating ...   

                                               input  output  \
0  A 42-year-old woman presents to the emergency ...       1   
1  A 69-year-old man presents to his dermatologis...       1   
2  A 64-year-old female with a long-standing hist...       1   
3  An endocrinologist is working with a pharmaceu...       1   
4  A pediatrician is investigating determinants o...       1   

                                        for_sampling  
0  {'input': 'A 42-year-old woman presents to the...  
1  {'input': 'A 69-year-old man presents to his d...  

In [10]:
import os
import pandas as pd
import numpy as np
import time
from sentence_transformers import SentenceTransformer
import faiss
from tqdm import tqdm
import random
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def mmr(query_embedding, doc_embeddings, diversity, top_n):
    faiss.normalize_L2(doc_embeddings)
    faiss.normalize_L2(query_embedding.reshape(1, -1))

    selected_indices = []
    candidate_indices = list(range(len(doc_embeddings)))

    for _ in range(top_n):
        if not candidate_indices:
            break

        if len(selected_indices) == 0:
            selected_idx = candidate_indices[np.argmax(np.dot(doc_embeddings[candidate_indices], query_embedding.T))]
        else:
            selected_embeddings = doc_embeddings[selected_indices]
            similarity_to_selected = np.dot(doc_embeddings[candidate_indices], selected_embeddings.T)
            diversity_scores = np.max(similarity_to_selected, axis=1)
            relevance_scores = np.dot(doc_embeddings[candidate_indices], query_embedding.T).flatten()
            mmr_scores = (1 - diversity) * relevance_scores - diversity * diversity_scores
            selected_idx = candidate_indices[np.argmax(mmr_scores)]

        selected_indices.append(selected_idx)
        candidate_indices.remove(selected_idx)

    return selected_indices

def mmr_sampling(dataframe, sampling_dict, embedding_model='all-MiniLM-L6-v2', batch_size=64, diversity=0.7, seed=42):
    set_seed(seed)

    print("Loading SentenceTransformer model on GPU...")
    model = SentenceTransformer(embedding_model, device="cuda:0")

    sampled_rows = []

    for dataset_name, sample_count in tqdm(sampling_dict.items(), desc="Processing datasets"):
        subset = dataframe[(dataframe['dataset'] == dataset_name) & (dataframe['split_data'] == 'test')].copy()
        num_rows = len(subset)
        if num_rows == 0:
            print(f"No data found for dataset: {dataset_name}")
            continue

        print(f"Generating embeddings for dataset: {dataset_name} ({num_rows} rows)")
        start_time = time.time()

        embeddings = model.encode(
            subset['for_sampling'].astype(str).tolist(),
            batch_size=batch_size,
            convert_to_numpy=True,
            show_progress_bar=True
        )

        elapsed_time = time.time() - start_time
        print(f"Embedding generation took {elapsed_time:.2f} seconds for {num_rows} rows")

        query_embedding = np.mean(embeddings, axis=0)

        print(f"Applying MMR for dataset: {dataset_name}")
        selected_indices = mmr(query_embedding, embeddings, diversity, sample_count)
        sampled_subset = subset.iloc[selected_indices]
        sampled_rows.append(sampled_subset)

    return pd.concat(sampled_rows, ignore_index=True)

sampling_config = {
    'medqa': 150,
    'medmcqa': 150,
    'pubmedqa': 1200, # 150 
    'bionli': 270, # 225
    'chemdner': 90,
    'ddi': 135,
    'meddialog': 600,
    'pubmed': 600,
}

set_seed(42)

# 샘플링 실행
sampled_test = mmr_sampling(diabetes3, sampling_config, batch_size=4096, diversity=0.7, seed=42)

# 결과 확인
print(sampled_test.head())
print(f"총 샘플링된 행 수: {len(sampled_test)}")

# 결과 저장
sampled_test.to_csv("final_combined_test_sample.csv")


Loading SentenceTransformer model on GPU...


Processing datasets:   0%|          | 0/8 [00:00<?, ?it/s]

Generating embeddings for dataset: medqa (185 rows)


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.62it/s]

Embedding generation took 0.19 seconds for 185 rows
Applying MMR for dataset: medqa


...

Generating embeddings for dataset: pubmed (1351 rows)


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.65s/it]


Embedding generation took 4.12 seconds for 1351 rows
Applying MMR for dataset: pubmed


Processing datasets: 100%|██████████| 8/8 [00:12<00:00,  1.51s/it]


  dataset split_data                                           features  \
0   medqa       test  {'question': 'A 67-year-old man presents to hi...   
1   medqa       test  {'question': "A 37-year-old woman presents to ...   
2   medqa       test  {'question': "Shortly after delivery, a female...   
3   medqa       test  {'question': 'A 64-year-old man with a history...   
4   medqa       test  {'question': 'A researcher is tracing the fate...   

                                               input  output  \
0  A 67-year-old man presents to his primary care...       1   
1  A 37-year-old woman presents to the emergency ...       1   
2  Shortly after delivery, a female newborn devel...       1   
3  A 64-year-old man with a history of type 2 dia...       1   
4  A researcher is tracing the fate of C-peptide,...       1   

                                        for_sampling  
0  {'input': 'A 67-year-old man presents to his p...  
1  {'input': 'A 37-year-old woman presents to the...  

In [1]:
import pandas as pd

final_combined_train_sample = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/final_combined_train_sample.csv")
final_combined_test_sample = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/final_combined_test_sample.csv")

In [2]:
# 중복 제거 후 겹치는 개수 확인
train_unique = final_combined_train_sample.drop_duplicates(subset='features')
test_unique = final_combined_test_sample.drop_duplicates(subset='features')

# train/test 겹치는 개수 계산
overlapping_count = len(pd.merge(train_unique, test_unique, on='features'))
print(f"Train/Test 겹치는 feature 개수: {overlapping_count}")


Train/Test 겹치는 feature 개수: 0


In [3]:
import numpy as np

final_combined_train_sample['task'] = None
# medqa
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'medqa', 'task'] = 'qa_objective_1'
# medmcqa
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'medmcqa', 'task'] = 'qa_objective_2'
# pubmedqa
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'pubmedqa', 'task'] = 'qa_objective_3'
# bionli
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'bionli', 'task'] = 'nli'
# chemdner
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'chemdner', 'task'] = 'ie'
# ddi
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'ddi', 'task'] = 're'
# meddialog
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'meddialog', 'task'] = 'generation'
# pubmed
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'pubmed', 'task'] = 'summarization'

In [4]:
import numpy as np

final_combined_test_sample['task'] = None
# medqa
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'medqa', 'task'] = 'qa_objective_1'
# medmcqa
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'medmcqa', 'task'] = 'qa_objective_2'
# pubmedqa
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'pubmedqa', 'task'] = 'qa_objective_3'
# bionli
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'bionli', 'task'] = 'nli'
# chemdner
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'chemdner', 'task'] = 'ie'
# ddi
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'ddi', 'task'] = 're'
# meddialog
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'meddialog', 'task'] = 'generation'
# pubmed
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'pubmed', 'task'] = 'summarization'

In [5]:
final_combined_train_sample.head(2)

,Unnamed: 0,dataset,split_data,features,input,output,for_sampling,task
0,0,medqa,train,{'question': 'A 42-year-old woman presents to ...,A 42-year-old woman presents to the emergency ...,1,{'input': 'A 42-year-old woman presents to the...,qa_objective_1
1,1,medqa,train,"{'question': ""A 69-year-old man presents to hi...",A 69-year-old man presents to his dermatologis...,1,"{'input': ""A 69-year-old man presents to his d...",qa_objective_1


In [6]:
import ast

def parse_features(row):
    try:
        return ast.literal_eval(row)
    except (ValueError, SyntaxError):
        return {}
    
def generate_input_output(row):
    instruction_value = ""; input_value = ""; output_value = ""

    task = row['task']
    features = parse_features(row['features'])
    if row['dataset']=='medqa' and task=='qa_objective_1':
        question = features.get('question', None)
        options = features.get('options', None)
        answer = features.get('answer', None)
        answer_idx = features.get('answer_idx', None)
        instruction_value = "Select the most appropriate answer for the given medical question from the provided options."
        input_value = (
            f"{question} Please select one of the following: A) {options['A']}, B) {options['B']}, C) {options['C']}, D) {options['D']}."
        )
        output_value = (
            f"{answer_idx}) {answer}"
        )
    elif row['dataset']=='medmcqa' and task=='qa_objective_2': 
        question = features.get('question', None)
        options = {
            'A': features.get('opa', '').strip(),
            'B': features.get('opb', '').strip(),
            'C': features.get('opc', '').strip(),
            'D': features.get('opd', '').strip(),
        }
        answer_num = features.get('cop', None)
        answer_idx = 'A' if answer_num == 0 else 'B' if answer_num == 1 else 'C' if answer_num == 2 else 'D'
        answer = features.get('opa', '').strip() if answer_num == 0 else features.get('opb', '').strip() if answer_num == 1 else features.get('opc', '').strip() if answer_num == 2 else features.get('opd', '').strip()
        instruction_value = "Select the most appropriate answer for the given medical question from the provided options."
        input_value = (
            f"{question} Please select one of the following: A) {options['A']}, B) {options['B']}, C) {options['C']}, D) {options['D']}."
        )
        output_value = (
            f"{answer_idx}) {answer}"
        )
    elif row['dataset']=='pubmedqa' and task=='qa_objective_3':
        question = features.get('QUESTION', '').strip()
        context = " ".join(features.get('CONTEXTS'))
        # answer = features.get('LONG_ANSWER', '').strip() 
        answer = features.get('final_decision', '').strip()
        instruction_value = "Choose the correct anser (Yes, No, or Maybe) for the given question based on the proviced context."
        input_value = (
            f"Question: {question} "
            f"Context: {context}"
        )
        output_value = f"{answer}"
    elif row['dataset']=='bionli' and task=='nli':
        query = features.get('query', '') 
        instruction_value =  "Please classify the relationship between the given premise and hypothesis into one of the following labels: entailment, contradiction, or neutral. return only the label."
        input_value = query.split("INPUT: ")[1].split("[HYP]")[0].strip() if "INPUT: " in query else None
        output_value = features.get('answer', None)
    elif row['dataset']=='chemdner' and task=='ie':
        instruction_value = "Extract all chemical entities (chemical compounds, drugs, and molecules) from the given text."
        input_value = features.get('text', None)
        answer = features.get('entities', None)
        if isinstance(answer, list) and len(answer) > 0:
            output_value = answer[0]
        else:
            output_value = ''
    elif row['dataset']== 'ddi' and task=='re':
        instruction_value = "Analyze the sentence with two drugs labeled as @DRUG_A$ and @DRUG_B$. Extract the interaction between @DRUG_A$ and @DRUG_B$ from the input sentence by selecting only one of the following options: 'DDI-effect', 'DDI-mechanism', 'DDI-advise', 'DDI-false', and 'DDI-int'. 'DDI-effect': Choose this when the interaction describes an effect or a pharmacodynamic mechanism. 'DDI-mechanism': Choose this for interactions explained by pharmacokinetic mechanisms. 'DDI-advise': Choose this when the sentence provides a recommendation or advice about the drug interaction. 'DDI-false': Choose this if there is no actual drug-drug interaction in the sentence. 'DDI-int': Choose this when a drug-drug interaction is mentioned without additional detail."
        input_value = features.get('conversations', [])[0].get('value', None).split("INPUT:")[1].split("OUTPUT:")[0].strip()
        output_value = features.get('conversations', [])[1].get('value', None)
    elif row['dataset']=='meddialog' and task=='generation':
        instruction_value = features.get('tgt', None)
        input_value = features.get('src', None).split("Doctor:")[0].replace("Patient:", "").strip()
        output_value = features.get('src', None).split("Doctor:")[1].strip()
    elif row['dataset']=='pubmed' and task=='summarization':
        instruction_value = "Summarize the key findings of the given PubMed abstract into structured fields: Objective, Methods, Results, and Conclusion."
        article = features.get('article', '').strip()  
        abstract = features.get('abstract', '').strip() 
        input_value = f"{article}"
        output_value = abstract
    return instruction_value, input_value, output_value

final_combined_train_sample[['instruction', 'input', 'output']] = final_combined_train_sample.apply(
    lambda row: pd.Series(generate_input_output(row)), axis=1
)

final_combined_test_sample[['instruction', 'input', 'output']] = final_combined_test_sample.apply(
    lambda row: pd.Series(generate_input_output(row)), axis=1
)

In [7]:
from sklearn.utils import resample
import pandas as pd
# pubmedqa

# trainset
df_train = final_combined_train_sample[final_combined_train_sample['dataset']=='pubmedqa']
df_yes = df_train[df_train['output']=='yes']
df_no = df_train[df_train['output']=='no']

df_downsampled = resample(df_yes, replace=False, n_samples=len(df_no), random_state=42)
balanced_df = pd.concat([df_downsampled, df_no])

final_df_balanced = final_combined_train_sample[final_combined_train_sample['dataset'] != 'pubmedqa']
final_combined_train_sample = pd.concat([final_df_balanced, balanced_df])

# testset
df_test = final_combined_test_sample[final_combined_test_sample['dataset']=='pubmedqa']
df_yes = df_test[df_test['output']=='yes']
df_no = df_test[df_test['output']=='no']

df_downsampled = resample(df_yes, replace=False, n_samples=len(df_no), random_state=42)
balanced_df = pd.concat([df_downsampled, df_no])

final_df_balanced = final_combined_test_sample[final_combined_test_sample['dataset'] != 'pubmedqa']
final_combined_test_sample = pd.concat([final_df_balanced, balanced_df])

In [8]:
from sklearn.utils import resample
import pandas as pd
# bionli

# trainset
df_train = final_combined_train_sample[final_combined_train_sample['dataset']=='bionli']
df_yes = df_train[df_train['output']=='contradiction']
df_no = df_train[df_train['output']=='entailment']

df_downsampled = resample(df_yes, replace=False, n_samples=len(df_no), random_state=42)
balanced_df = pd.concat([df_downsampled, df_no])

final_df_balanced = final_combined_train_sample[final_combined_train_sample['dataset'] != 'bionli']
final_combined_train_sample = pd.concat([final_df_balanced, balanced_df])

# testset
df_test = final_combined_test_sample[final_combined_test_sample['dataset']=='bionli']
df_yes = df_test[df_test['output']=='contradiction']
df_no = df_test[df_test['output']=='entailment']

df_downsampled = resample(df_yes, replace=False, n_samples=len(df_no), random_state=42)
balanced_df = pd.concat([df_downsampled, df_no])

final_df_balanced = final_combined_test_sample[final_combined_test_sample['dataset'] != 'bionli']
final_combined_test_sample = pd.concat([final_df_balanced, balanced_df])

In [9]:
final_df = pd.concat([final_combined_train_sample, final_combined_test_sample])

In [11]:
final_combined_train_sample.to_json("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/train_instruction_dataset.json", orient="columns", indent=4)
final_combined_test_sample.to_json("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/test_instruction_dataset.json", orient="columns", indent=4)

In [12]:
final_combined_train_sample.head(2)

,Unnamed: 0,dataset,split_data,features,input,output,for_sampling,task,instruction
0,0,medqa,train,{'question': 'A 42-year-old woman presents to ...,A 42-year-old woman presents to the emergency ...,D) Ursodeoxycholic acid,{'input': 'A 42-year-old woman presents to the...,qa_objective_1,Select the most appropriate answer for the giv...
1,1,medqa,train,"{'question': ""A 69-year-old man presents to hi...",A 69-year-old man presents to his dermatologis...,B) Immunosuppression,"{'input': ""A 69-year-old man presents to his d...",qa_objective_1,Select the most appropriate answer for the giv...
